In [1]:
import os
from google.oauth2.service_account import Credentials

# Get the path from the environment variable
json_path = os.getenv("GOOGLE_APPLICATION_CREDENTIALS")

print(json_path)

./secrets/gspread-for-ipynb-tiktok-3adba504e647.json
